In [1]:
import os

# Retrieve the API key from the environment variable
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Confirm that the API key is set (do not print the key in production code)
if os.environ["OPENAI_API_KEY"]:
    print("API key is set.")
else:
    print("API key is not set.")


API key is set.


In [2]:
pip install lightrag-hku

ERROR: Could not find a version that satisfies the requirement lightrag-hku (from versions: none)
ERROR: No matching distribution found for lightrag-hku
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from lightrag import LightRAG, QueryParam
from lightrag.llm import gpt_4o_mini_complete, gpt_4o_complete

In [ ]:
import os
import sys
import shutil
from transformers import AutoModel,AutoTokenizer

# Define the path of the zip and the extraction path

extraction_path = "/home/manohargolleru2000/server/Notebook/"


# Initialize LightRAG with the working directory
rag = LightRAG(
    working_dir=extraction_path,
    llm_model_func=gpt_4o_mini_complete
)

print("RAG system initialized with restored data.")


In [ ]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
import time

# Initialize conversation history to store selected pairs
conversation_history = []

# Get user input for the query
user_query = input("Enter your query: ")

# Function to handle queries and provide options
def generate_response_options(query_mode, user_query):
    start_time = time.time()
    response = rag.query(
        f"As Todd, respond to the following question in a conversational manner, keeping the response under 15 words for brevity and relevance: {user_query}. Focus on providing an honest and personal answer that aligns with my perspective in the story.",
        param=QueryParam(mode=query_mode)
    )
    latency = time.time() - start_time
    return response, latency

# Generate responses for each mode
response_naive, latency_naive = generate_response_options("naive", user_query)
response_local, latency_local = generate_response_options("local", user_query)
response_global, latency_global = generate_response_options("global", user_query)
response_hybrid, latency_hybrid = generate_response_options("hybrid", user_query)

# Display the options to the user
print(f"Naive: {response_naive} (Latency: {latency_naive:.2f} seconds)")
print(f"Local: {response_local} (Latency: {latency_local:.2f} seconds)")
print(f"Global: {response_global} (Latency: {latency_global:.2f} seconds)")
print(f"Hybrid: {response_hybrid} (Latency: {latency_hybrid:.2f} seconds)")

# Get user's choice of response
selected_response = input("Select the best response (naive, local, global, hybrid): ").strip().lower()

# Map user's choice to the corresponding response
response_mapping = {
    "naive": response_naive,
    "local": response_local,
    "global": response_global,
    "hybrid": response_hybrid
}
chosen_response = response_mapping.get(selected_response, "No valid choice made.")

# Store the user query and chosen response pair in conversation history
conversation_history.append((user_query, chosen_response))

print(f"You selected: {chosen_response}")

# The conversation history can now be used to inform future queries.
# Example: printing the conversation history
print("\nConversation History:")
for i, (q, r) in enumerate(conversation_history, start=1):
    print(f"{i}. User: {q}\n   Todd: {r}")
